#### Import libraries

In [1]:

import sys
sys.path.append('BEHRT/Early_integration/')
sys.path.append('BEHRT/')


from Utils.common import *
from Utils.pytorch import load_model
from Utils.utils import age_vocab, input_vocab, mod_vocab, delay_vocab
from Utils.optimiser import adam
import random
import pytorch_pretrained_bert as Bert
from torch.utils.data import DataLoader
import pandas as pd
import sklearn.metrics as skm
import numpy as np
import torch
import torch.nn as nn
import os
import time
import warnings
warnings.filterwarnings(action='ignore')
from torch.utils.data.dataset import Dataset
from Utils.dataLoader_utils import seq_padding, random_mask, code2index,label2index, position_idx, index_seg
from Utils.add_endpoints import add_endp
from Utils.handle_file import handle_file

hf = handle_file()
from Utils.preprocessing import cal_age, cal_therapies, spark_init, cal_subtherapies, int_to_list, flatten_list, int_to_str, cal_therapies_fr, divide_frame

from datetime import datetime, timedelta
from torch.utils.data import Dataset, DataLoader, Sampler
import sklearn
from sklearn.model_selection import StratifiedKFold
import math
import datetime 
from sklearn.metrics import roc_auc_score
from Utils import optimiser
from Utils.dataLoader_utils import ImbSampler, OverSampler, StratifiedSampler
from Utils.NextXVisit_v2 import NextVisit
from Models.BertForClassification import BertForClassification
import warnings
warnings.filterwarnings(action='ignore')
from collections import Counter
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


2023-03-28 04:47:15.593041: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 04:47:16.746643: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-28 04:47:16.746762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-28 04:47:16.746774: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like t

#### Utils 

In [ ]:

import torch
from torch import nn
import numpy as np
import math

class NextVisit(Dataset):
    def __init__(self, token2idx, label2idx, dataframe,  max_len, patid='patid', code='inputs', age='age', label = 'label', bio='biological features', delay = 'delays_in_months', NPI='NPI'):
        # dataframe preproecssing
        # filter out the patient with number of visits less than min_visit
        self.vocab = token2idx
        self.label_vocab = label2idx
        self.max_len = max_len
        self.code = dataframe[code]
        self.age = dataframe[age]
        self.label = dataframe[label]
        self.patid = dataframe[patid]
        self.bio = dataframe[bio]
        self.NPI = dataframe[NPI]
        self.delay = dataframe[delay]
     
        

    def __getitem__(self, index):
        """
        return: age, code, position, segmentation, mask, label
        """
        # cut data
        age = self.age[index]
        code = self.code[index]
        label = self.label[index]
        patid = self.patid[index]
        delay = self.delay[index]
        bio = self.bio[index]
        NPI = self.NPI[index]
        
        # change id type to list
        patid = [patid]

        # extract data
        age = age[(-self.max_len+1):]
        code = code[(-self.max_len+1):]
        delay = delay[(-self.max_len+1):]
        bio = bio[(-self.max_len+1):]
        NPI = NPI[(-self.max_len+1):]

        # mask 0:len(code) to 1, padding to be 0
        mask = np.ones(self.max_len)
        mask[len(code):] = 0

        # pad age seque once and code sequence
        age = seq_padding(age, self.max_len, symbol=0)
        delay = seq_padding(delay, self.max_len, symbol=0)
        bio = seq_padding(bio, self.max_len, symbol=[0]*5)
        NPI = seq_padding(NPI, self.max_len, symbol=0)
        
        
        tokens, code = code2index(code, self.vocab)
        _, label = label2index(label, self.label_vocab)

        # get position code and segment code
        tokens = seq_padding(tokens, self.max_len)
        position = position_idx(tokens)
        segment = index_seg(tokens)

        # pad code and label
        code = seq_padding(code, self.max_len, symbol=self.vocab['PAD'])
      #  label = seq_padding(label, self.max_len, symbol=-1)
        

        return torch.LongTensor(age).to(torch.float32), torch.LongTensor(code), torch.LongTensor(bio).to(torch.float32), torch.LongTensor(delay).to(torch.float32), torch.LongTensor(NPI).to(torch.float32), \
                torch.LongTensor(position), torch.LongTensor(segment), \
               torch.LongTensor(mask), torch.LongTensor(label), torch.LongTensor(patid)

    def __len__(self):
        return len(self.code)
    
    
 
class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, segment, age
    """

    def __init__(self, config, feature_dict=None):
        super(BertEmbeddings, self).__init__()

        if feature_dict is None:
            self.feature_dict = {
                'word': True,
                'bio_features' : True,
                'seg': True,
                'age': True,
                'delays':True,
                'NPI':True,
                'position': True,
            }
        else:
            self.feature_dict = feature_dict

        if feature_dict['word']:
            self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)

        if feature_dict['seg']:
            self.segment_embeddings = nn.Embedding(config.seg_vocab_size, config.hidden_size)
        
        if feature_dict['bio_features']:
            self.bio_embeddings = LinearActivation(config.bio_vocab_size, config.hidden_size)
            
        if feature_dict['age']:
            self.age_embeddings = SineActivation(config.age_vocab_size, config.hidden_size)#.  \
         #   from_pretrained(self.SineActivation(config.age_vocab_size, config.hidden_size))
            
        if feature_dict['delays']:
            self.delays_embeddings = SineActivation(config.delay_vocab_size, config.hidden_size)#. \
            #from_pretrained(self.SineActivation(config.delay_vocab_size, config.hidden_size))

        if feature_dict['NPI']:
            self.NPI_embeddings = LinearActivation(config.NPI_vocab_size, config.hidden_size)

        if feature_dict['position']:
            self.posi_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size). \
            from_pretrained(embeddings=self._init_posi_embedding(config.max_position_embeddings, config.hidden_size))

        self.LayerNorm = Bert.modeling.BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, word_ids, bio_ids, age_ids, delays_ids, seg_ids, posi_ids, NPI_ids):
        embeddings = self.word_embeddings(word_ids)
        
      #  print(delays_ids)
        if self.feature_dict['seg']:
            segment_embed = self.segment_embeddings(seg_ids)
            embeddings = embeddings + segment_embed
        
        if self.feature_dict['bio_features']:
            bio_embed = self.bio_embeddings(bio_ids)
            embeddings = embeddings + bio_embed
            
        if self.feature_dict['age']:
       
            age_embed = self.age_embeddings(age_ids)
            embeddings = embeddings + age_embed
          #  embeddings = torch.tensordot(embeddings, age_embed)
            
        if self.feature_dict['delays']:
            delays_embed = self.delays_embeddings(delays_ids)
            embeddings = embeddings + delays_embed
         #   embeddings = torch.tensordot(embeddings, delays_embed)
            
        
        if self.feature_dict['NPI']:
            NPI_embed = self.NPI_embeddings(NPI_ids)
            embeddings = embeddings + NPI_embed
           # embeddings = torch.tensordot(embeddings, NPI_embed)
            

        if self.feature_dict['position']:
            posi_embeddings = self.posi_embeddings(posi_ids)
            embeddings = embeddings + posi_embeddings
        
    
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings      
    
      
    def SineActivation(self, in_features, out_features, max_position_embedding= 512):
       
        w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        b0 = nn.parameter.Parameter(torch.randn(1))
        w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        b = nn.parameter.Parameter(torch.randn(out_features-1))
        f = torch.sin
        
        def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
            if arg:
                v1 = f(torch.matmul(tau, w) + b, arg)
            else:
                #print(w.shape, t1.shape, b.shape)
                v1 = f(torch.matmul(tau, w) + b)
            v2 = torch.matmul(tau, w0) + b0
            #print(v1.shape)
            return torch.cat([v1, v2], -1)
        
        tau = np.ones(max_position_embedding, dtype=np.float32)
        out_list = list()
        
        for pos in tau: 
            pos = torch.tensor(pos.reshape(-1,1))
            out = t2v(pos, f, out_features, w, b, w0, b0)
            out_list.append(torch.flatten(out))
        return torch.stack(out_list, 0)
    

    def _init_posi_embedding(self, max_position_embedding, hidden_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / hidden_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / hidden_size)))

        # initialize position embedding table
        lookup_table = np.zeros((max_position_embedding, hidden_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(0, hidden_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(1, hidden_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)
        
def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], -1)

class SineActivation(nn.Module):
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(out_features-1))
        self.f = torch.sin
      #  self.fc1 = nn.Linear(out_features, out_features)


    def forward(self, tau):
        out_list = list()
        for pos in tau: 
            pos = torch.tensor(np.expand_dims(pos.cpu(), axis=1))
            out = t2v(pos.to(global_params['device']), self.f, self.out_features, self.w, self.b, self.w0, self.b0)
        #    out = self.fc1(x)
            out_list.append(out)

        return torch.stack(out_list, 0)

class LinearActivation(nn.Module):
    def __init__(self, in_features, hidden_size):
        super(LinearActivation, self).__init__()
        self.in_features = in_features
        self.hidden_size = hidden_size
        
    def forward(self, inp):
        out_list = list()
        LinearLayer = nn.Linear(self.in_features, self.hidden_size)
        LinearLayer = LinearLayer.to(global_params['device'])   ### move layer to device because all the inputs are in the gpu
       
        for idx in inp:
          
          ## if dim of idx is >= than 1 -> when number of biological features is more than one
            try:   
                out = LinearLayer(idx)
            except RuntimeError:
               # print('linearlayer {}'.format(LinearLayer.weight.dtype))
                 ## if we need to expand dimension move it to cpu first then remove it to gpu (next line)
                out = LinearLayer(idx.reshape(-1,1).to(global_params['device']))
            out_list.append(out)
        return torch.stack(out_list, 0)

    
class BertModel(Bert.modeling.BertPreTrainedModel):
    def __init__(self, config, feature_dict,  add_pooling_layer=True):
        super(BertModel, self).__init__(config)
        self.config = config
        self.embeddings = BertEmbeddings(config=config, feature_dict = feature_dict)
        self.encoder = BertEncoder(config=config)
        self.pooler = BertPooler(config)
        self.apply(self.init_bert_weights)
       

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value
        


    def forward(self, input_ids, bio_ids, age_ids, delays_ids, seg_ids, posi_ids,  NPI_ids, attention_mask, output_attentions= None, output_hidden_states = None, return_dict = None)-> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions 
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, bio_ids, age_ids, delays_ids, seg_ids, posi_ids, NPI_ids)
        encoder_outputs = self.encoder(embedding_output,
                                      attention_mask = extended_attention_mask, output_attentions = output_attentions, output_hidden_states = output_hidden_states, return_dict = return_dict)
        
       # sequence_output = encoded_layers[-1]
         
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None 
        
     #   if not output_all_encoded_layers:
    #        encoded_layers = encoded_layers[-1]
            
        if not return_dict: 
            return (sequence_output, pooled_output) + encoded_layers[1:]
         #   return encoded_layers, pooled_output

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values = encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


class BertForClassification(Bert.modeling.BertPreTrainedModel):
    def __init__(self, config,  num_labels, feature_dict, class_weights):
        super(BertForClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config, feature_dict)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.class_weights = class_weights
        self.apply(self.init_bert_weights)
        
    def forward(self, input_ids=None, bio_ids=None, age_ids=None, delays_ids = None, seg_ids=None, posi_ids=None,  NPI_ids =None, attention_mask=None, labels=None, output_attentions = True, output_hidden_states = True, return_dict = True)-> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        outputs = self.bert(input_ids, bio_ids, age_ids, delays_ids, seg_ids, posi_ids,  NPI_ids , attention_mask,  output_attentions = output_attentions, output_hidden_states=output_hidden_states, return_dict = return_dict)
       
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        

        if labels is not None:
            
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.class_weights)
            loss = loss_fct(logits, labels)#.unsqueeze(1))
  
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
    
def calculate_pos_weights(class_counts):
    pos_weights = np.ones_like(class_counts)
    neg_counts = [len(data)-pos_count for pos_count in class_counts]
    for cdx, pos_count, neg_count in enumerate(zip(class_counts,  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5)

    return torch.as_tensor(pos_weights, dtype=torch.float)

In [ ]:
import torch
import torch.nn as nn
import pytorch_pretrained_bert as Bert
import numpy as np
from typing import Optional, Tuple, Union
from transformers.models.bert.modeling_bert import *
from transformers.modeling_outputs import SequenceClassifierOutput, BaseModelOutputWithPoolingAndCrossAttentions
import numpy as np
from torch.utils.data.dataset import Dataset
#P/¨%from dataLoader.utils import seq_padding,code2index,label2index, position_idx, index_seg
import torch


class NextVisit(Dataset):
    def __init__(self, token2idx, label2idx, mod2idx, age2idx, del2idx, NPI2idx, dataframe,  max_len, patid='patid', code='inputs', age='age', label = 'label', mod = 'modalities', delay = 'delays_in_months', NPI='NPI6'):
        # dataframe preproecssing
        # filter out the patient with number of visits less than min_visit
        self.vocab = token2idx
        self.mod2idx = mod2idx
        self.age2idx = age2idx
        self.del2idx = del2idx
        self.NPI2idx = NPI2idx
        self.label_vocab = label2idx
        self.max_len = max_len
        self.code = dataframe[code]
        self.age = dataframe[age]
        self.label = dataframe[label]
        self.patid = dataframe[patid]
        self.modalities = dataframe[mod]
        self.NPI = dataframe[NPI]
        self.delay = dataframe[delay]
     
        

    def __getitem__(self, index):
        """
        return: age, code, position, segmentation, mask, label
        """
        # cut data
        age = self.age[index]
        code = self.code[index]
        label = self.label[index]
        patid = self.patid[index]
        delay = self.delay[index]
        modalities = self.modalities[index]
        NPI = self.NPI[index]
        
        # change id type to list
        patid = [patid]

        # extract data
        age = age[(-self.max_len+1):]
        code = code[(-self.max_len+1):]
        delay = delay[(-self.max_len+1):]
        modalities = modalities[(-self.max_len+1):]
        NPI = NPI[(-self.max_len+1):]

        # mask 0:len(code) to 1, padding to be 0
        mask = np.ones(self.max_len)
        mask[len(code):] = 0

        # pad age seque once and code sequence
        age = seq_padding(age, self.max_len, token2idx=self.age2idx)
        delay = seq_padding(delay, self.max_len, token2idx=self.del2idx)
        modalities = seq_padding(modalities, self.max_len, token2idx=self.mod2idx)
        NPI = seq_padding(NPI, self.max_len, token2idx=self.NPI2idx)
      
        tokens, code = code2index(code, self.vocab)
        _, label = label2index(label, self.label_vocab)

        # get position code and segment code
        tokens = seq_padding(tokens, self.max_len)
        position = position_idx(tokens)
        segment = index_seg(tokens)

        # pad code and label
        code = seq_padding(code, self.max_len, symbol=self.vocab['PAD'])
      #  label = seq_padding(label, self.max_len, symbol=-1)
        

        return torch.LongTensor(age), torch.LongTensor(code), torch.LongTensor(modalities), torch.LongTensor(delay), torch.LongTensor(NPI), \
                torch.LongTensor(position), torch.LongTensor(segment), \
               torch.LongTensor(mask), torch.LongTensor(label), torch.LongTensor(patid)

    def __len__(self):
        return len(self.code)
    
    
 
class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, segment, age
    """

    def __init__(self, config, feature_dict=None):
        super(BertEmbeddings, self).__init__()

        if feature_dict is None:
            self.feature_dict = {
                'word': True,
                'modalities' : True,
                'seg': True,
                'age': True,
                'delays':True,
                'NPI':True,
                'position': True,
            }
        else:
            self.feature_dict = feature_dict

        if feature_dict['word']:
            self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)

        if feature_dict['seg']:
            self.segment_embeddings = nn.Embedding(config.seg_vocab_size, config.hidden_size)
        
        if feature_dict['modalities']:
            self.modalities_embeddings = nn.Embedding(config.modalities_vocab_size, config.hidden_size)

        if feature_dict['age']:
            self.age_embeddings = nn.Embedding(config.age_vocab_size, config.hidden_size)
            
        if feature_dict['delays']:
            self.delays_embeddings = nn.Embedding(config.delays_vocab_size, config.hidden_size)

        if feature_dict['NPI']:
            self.NPI_embeddings = nn.Embedding(config.NPI_vocab_size, config.hidden_size)

        if feature_dict['position']:
            self.posi_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size). \
            from_pretrained(embeddings=self._init_posi_embedding(config.max_position_embeddings, config.hidden_size))

        self.LayerNorm = Bert.modeling.BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
    

    def forward(self, word_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids, NPI_ids):
        embeddings = self.word_embeddings(word_ids)
        
        if self.feature_dict['seg']:
            segment_embed = self.segment_embeddings(seg_ids)
            embeddings = embeddings + segment_embed
        
        if self.feature_dict['modalities']:
            modalities_embed = self.modalities_embeddings(modalities_ids)
            embeddings = embeddings + modalities_embed

        if self.feature_dict['age']:
            age_embed = self.age_embeddings(age_ids)
            embeddings = embeddings + age_embed
            
        if self.feature_dict['delays']:
            delays_embed = self.delays_embeddings(delays_ids)
            embeddings = embeddings + delays_embed
        
        if self.feature_dict['NPI']:
            NPI_embed = self.delays_embeddings(NPI_ids)
            embeddings = embeddings + NPI_embed

        if self.feature_dict['position']:
            posi_embeddings = self.posi_embeddings(posi_ids)
            embeddings = embeddings + posi_embeddings

        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings        

    def _init_posi_embedding(self, max_position_embedding, hidden_size):
        def even_code(pos, idx):
            return np.sin(pos / (10000 ** (2 * idx / hidden_size)))

        def odd_code(pos, idx):
            return np.cos(pos / (10000 ** (2 * idx / hidden_size)))

        # initialize position embedding table
        lookup_table = np.zeros((max_position_embedding, hidden_size), dtype=np.float32)

        # reset table parameters with hard encoding
        # set even dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(0, hidden_size, step=2):
                lookup_table[pos, idx] = even_code(pos, idx)
        # set odd dimension
        for pos in range(max_position_embedding):
            for idx in np.arange(1, hidden_size, step=2):
                lookup_table[pos, idx] = odd_code(pos, idx)

        return torch.tensor(lookup_table)
    

class BertModel(Bert.modeling.BertPreTrainedModel):
    def __init__(self, config, feature_dict,  add_pooling_layer=True):
        super(BertModel, self).__init__(config)
        self.config = config
        self.embeddings = BertEmbeddings(config=config, feature_dict = feature_dict)
        self.encoder = BertEncoder(config=config)
        self.pooler = BertPooler(config)
        self.apply(self.init_bert_weights)
       

    def get_input_embeddings(self):
        return self.embeddings.word_embeddings

    def set_input_embeddings(self, value):
        self.embeddings.word_embeddings = value
        


    def forward(self, input_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids,  NPI_ids, attention_mask, output_attentions= None, output_hidden_states = None, return_dict = None)-> Union[Tuple[torch.Tensor], BaseModelOutputWithPoolingAndCrossAttentions]:
        
        output_attentions = output_attentions if output_attentions is not None else self.config.output_attentions 
        output_hidden_states = (
            output_hidden_states if output_hidden_states is not None else self.config.output_hidden_states
        )
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        

        # We create a 3D attention mask from a 2D tensor mask.
        # Sizes are [batch_size, 1, 1, to_seq_length]
        # So we can broadcast to [batch_size, num_heads, from_seq_length, to_seq_length]
        # this attention mask is more simple than the triangular masking of causal attention
        # used in OpenAI GPT, we just need to prepare the broadcast dimension here.
        extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

        # Since attention_mask is 1.0 for positions we want to attend and 0.0 for
        # masked positions, this operation will create a tensor which is 0.0 for
        # positions we want to attend and -10000.0 for masked positions.
        # Since we are adding it to the raw scores before the softmax, this is
        # effectively the same as removing these entirely.
        extended_attention_mask = extended_attention_mask.to(dtype=next(self.parameters()).dtype)  # fp16 compatibility
        extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0

        embedding_output = self.embeddings(input_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids, NPI_ids)
        encoder_outputs = self.encoder(embedding_output,
                                      attention_mask = extended_attention_mask, output_attentions = output_attentions, output_hidden_states = output_hidden_states, return_dict = return_dict)
        
       # sequence_output = encoded_layers[-1]
         
        sequence_output = encoder_outputs[0]
        pooled_output = self.pooler(sequence_output) if self.pooler is not None else None 
        
     #   if not output_all_encoded_layers:
    #        encoded_layers = encoded_layers[-1]
            
        if not return_dict: 
            return (sequence_output, pooled_output) + encoded_layers[1:]
         #   return encoded_layers, pooled_output

        return BaseModelOutputWithPoolingAndCrossAttentions(
            last_hidden_state=sequence_output,
            pooler_output=pooled_output,
            past_key_values = encoder_outputs.past_key_values,
            hidden_states=encoder_outputs.hidden_states,
            attentions=encoder_outputs.attentions,
            cross_attentions=encoder_outputs.cross_attentions,
        )


class BertForClassification(Bert.modeling.BertPreTrainedModel):
    def __init__(self, config,  num_labels, feature_dict, class_weights):
        super(BertForClassification, self).__init__(config)
        self.num_labels = num_labels
        self.bert = BertModel(config, feature_dict)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)
        self.class_weights = class_weights
        self.apply(self.init_bert_weights)
        

    def forward(self, input_ids=None, modalities_ids = None, age_ids=None, delays_ids = None, seg_ids=None, posi_ids=None,  NPI_ids =None, attention_mask=None, labels=None, output_attentions = True, output_hidden_states = True, return_dict = True)-> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        
        outputs = self.bert(input_ids, modalities_ids, age_ids, delays_ids, seg_ids, posi_ids,  NPI_ids , attention_mask,  output_attentions = output_attentions, output_hidden_states=output_hidden_states, return_dict = return_dict)
       
        pooled_output = outputs[1]
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        

        if labels is not None:
            
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.class_weights)
            loss = loss_fct(logits, labels)#.unsqueeze(1))
  
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )
    
def calculate_pos_weights(class_counts):
    pos_weights = np.ones_like(class_counts)
    neg_counts = [len(data)-pos_count for pos_count in class_counts]
    for cdx, pos_count, neg_count in enumerate(zip(class_counts,  neg_counts)):
        pos_weights[cdx] = neg_count / (pos_count + 1e-5)

    return torch.as_tensor(pos_weights, dtype=torch.float)

In [3]:
## Plot results
def display_training_stats(data):
    import pandas as pd

    
    # Create a DataFrame from our training statistics.
    df_stats = pd.DataFrame(data=data)
    
    # Display floats with two decimal places.
 #   pd.set_option('Average Train f1 score', 2)
 #   pd.set_option('Average Valid f1 score', 2)
 #   
    # Use the 'epoch' as the row index.
    df_stats = df_stats.set_index('epoch')
    
 #   hf._dump_pkl(df_stats, 'training_stats')    
    # Display the table.
    return df_stats

def cal_mean(df_stats, feature, epochs):
    return [np.mean(df_stats.loc[x, feature]) for x in range(1,epochs+1)]

def cal_std(df_stats, feature, epochs):
    return [np.std(df_stats.loc[x, feature]) for x in range(1,epochs+1)]
     

def plot_history(training_stats, epochs):
    
    sn.set(style="whitegrid")
    fig, ((ax1, ax2)) = plt.subplots(1,2, figsize = (20,7))
    
    mean_f1_train = cal_mean(display_training_stats(training_stats), 'Average Train f1 score', epochs)
    std_f1_train = cal_std(display_training_stats(training_stats), 'Average Train f1 score', epochs)
    
    mean_f1_valid = cal_mean(display_training_stats(training_stats), 'Average Valid f1 score', epochs)
    std_f1_valid = cal_std(display_training_stats(training_stats), 'Average Valid f1 score', epochs)
    
    mean_loss_train = cal_mean(display_training_stats(training_stats), 'Training Loss', epochs)
    std_loss_train = cal_std(display_training_stats(training_stats), 'Training Loss', epochs)
    
    mean_loss_valid = cal_mean(display_training_stats(training_stats), 'Validation Loss', epochs)
    std_loss_valid = cal_std(display_training_stats(training_stats), 'Validation Loss', epochs)
    
    
    
    ax1.plot(range(epochs), mean_f1_train, 'b-', label='Train f1 score')
    ax1.fill_between(range(epochs), np.array(mean_f1_train) - np.array(std_f1_train), np.array(mean_f1_train) + np.array(std_f1_train), color='b', alpha=0.2)
    ax1.plot(range(epochs), mean_f1_valid, 'r--', label='Valid f1 score')
    ax1.fill_between(range(epochs), np.array(mean_f1_valid) - np.array(std_f1_valid), np.array(mean_f1_valid) + np.array(std_f1_valid), color='r', alpha=0.2)
   
    ax2.plot(range(epochs), mean_loss_train, 'b-', label='Train loss')
    ax2.fill_between(range(epochs), np.array(mean_loss_train) - np.array(std_loss_train), np.array(mean_loss_train) + np.array(std_loss_train), color='b', alpha=0.2)
    ax2.plot(range(epochs), mean_loss_valid, 'r--', label='Valid loss')
    ax2.fill_between(range(epochs), np.array(mean_loss_valid) - np.array(std_loss_valid), np.array(mean_loss_valid) + np.array(std_loss_valid), color='r', alpha=0.2)
   
    ax1.grid()
    ax2.grid()
    
    plt.legend()
    plt.show()
    return


def precision_test(output, label):
    
  #  sig = nn.Sigmoid()
  #  output=sig(logits)
    label=np.argmax(label, axis=1)#.flatten().numpy()
    output=np.argmax(output, axis=1)#.flatten().numpy()
  #  output = np.argmax(np.where(output >= 0.5, 1, 0), axis=1).flatten()
    roc = sklearn.metrics.roc_auc_score(label ,output, average='macro')
    f1 =  sklearn.metrics.f1_score(label, output, average="macro")
    recall = sklearn.metrics.recall_score(label, output, average="macro")
    precision = sklearn.metrics.precision_score(label, output, average="macro")
 
    return f1, roc, recall, precision, output, label


def plot_roc_auc(fpr, tpr, roc_auc):
    
    import matplotlib.pyplot as plt
    import sklearn.metrics as metrics
    
   
    sn.set_style("whitegrid")
    fig,ax = plt.subplots(figsize=(13, 9))
    
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, '#067062', label = 'AUC = %0.2f' % roc_auc)

    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    #plt.savefig('ROC_12.png')
    plt.show()

def cm(true, predicted, fig_name, cmap = 'Purples'):
    
    cm = confusion_matrix(true, predicted)

    df_cm = pd.DataFrame(cm)
    fig = plt.figure(figsize=(10,7))
    try:
        heatmap = sn.heatmap(df_cm, annot=True, cmap=cmap)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=14)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=14)
    plt.ylabel('True labels')
    plt.xlabel('Predicted labels')
   #fig.savefig(fig_name+'.png')
    return 

def print_report(model, data): 
     
    y = []
    y_label = []
    total_f1, total_roc, total_precision, total_recall = [], [], [], []
    model.eval()


    for step, batch in enumerate(data):


        age_ids, input_ids, mod_ids, del_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
       
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)

        age_ids = age_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        del_ids = del_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])

        with torch.no_grad():
            outputs = model(input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, NPI_ids, attention_mask=attMask, labels=targets, output_attentions=True)
        

        logits = outputs.logits.cpu()
        targets = targets.cpu()

        y_label.append(targets)
        y.append(logits)
        
    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)

     # Compute ROC curve and ROC area for each class
    f1, roc, recall, precision, output, label = precision_test(y, y_label)

    # Compute micro-average ROC curve and ROC area
    label = np.argmax(y_label, axis=1)
    sig = nn.Sigmoid()
    output=sig(y)

    fpr, tpr, _ = sklearn.metrics.roc_curve(label, output[:,1])
 #   fpr, tpr, _ = sklearn.metrics.roc_curve(label, y_label[:,1])
    roc_auc = sklearn.metrics.auc(fpr, tpr)

    ## Plot ##  
    target_names = ['Non relapse', 'Relapse']

    print("f1 score is {:.4f}, Precision is {:.4f}, Recall is {:.4f}".format(f1, precision, recall))
    print(classification_report(label,  np.argmax(output, axis=1), target_names=target_names))
   # cm(label,  np.argmax(np.where(output >= 0.5, 1, 0), axis=1), 'cm')
    cm(label, np.argmax(output, axis=1), 'cm')
    
    plot_roc_auc(fpr, tpr, roc_auc)
  #  probs = F.softmax(y, dim=1).cpu().numpy()
  #  return probs, label
    return y_label, y, fpr, tpr, roc_auc  


def load_data(data, config, sampling = True, X_tr=None , y_tr = None):
    Dset = NextVisit(token2idx=tokenVocab, label2idx=labelVocab, mod2idx=modalitiesVocab, age2idx=ageVocab, del2idx=delayVocab, NPI2idx=NPIVocab, dataframe= data, max_len=global_params['max_len_seq'], code='inputs_normal_range_preprocessed_100_therap_removed', delay ='delays_100_therap_removed',
                         age = "age_100_therap_removed", mod ='modalities_100_therap_removed', NPI= 'NPI6')
    if sampling:
        return DataLoader(dataset= Dset, batch_size=config.batch_size,  sampler = StratifiedSampler(X_tr, y_tr, batch_size=config.batch_size))
    else:
        return DataLoader(dataset= Dset, batch_size=config.batch_size, shuffle=False)
        

In [5]:
def evaluate_roc(probs, y_true):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    preds = probs[:, 1]
    fpr, tpr, threshold = sklearn.metrics.roc_curve(y_true, preds)
    roc_auc = sklearn.metrics.auc(fpr, tpr)
    print(f'AUC: {roc_auc:.4f}')
       
    # Get accuracy over the test set
    y_pred = np.where(preds >= 0.9, 1, 0)
    accuracy = sklearn.metrics.f1_score(y_true, y_pred, average="macro")
   # accuracy = accuracy_score(y_true, y_pred)
    print(f'Accuracy: {accuracy*100:.2f}%')
    
    # Plot ROC AUC
    plt.title('Receiver Operating Characteristic')
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()

#### Performing checks for the resources available

In [4]:
""" Check if the gpu is available """
if torch.cuda.is_available():
   
    device = torch.device("cuda")
    print("There are %d GPU(s) available. " %torch.cuda.device_count())
    print("We will use the GPU: {}".format(torch.cuda.get_device_name(0)))
    
else:
    print("No GPU available, using the CPU instead")
    device = torch.device("cpu")
    

There are 1 GPU(s) available. 
We will use the GPU: Tesla V100-PCIE-32GB


In [7]:
""" Empty the cache to enable the use of the gpu """

def empty_cuda():
    torch.cuda.empty_cache()
    print(torch.cuda.memory_summary(device=None, abbreviated=False))


In [6]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
import wandb


os.environ['https_proxy'] = ""
os.environ['http_proxy'] = ""
os.environ['no_proxy']= ""
os.environ['HTTPS_PROXY']= ""
os.environ['HTTP_PROXY']= ""
os.environ['NO_PROXY']= ""

os.environ['WANDB_NOTEBOOK_NAME'] = 'BEHRT/Early_integration/Tasks/Binary_clf_3y.ipynb'
os.environ['WANDB_API_KEY'] = "4944e153a8b0f45a64d749b14d6916d253a54426"
os.environ['WANDB_NAME'] = "Binary_clf_npi"
os.environ['WANDB_NOTES'] = "Hyperparameters tuning using wandb for binary clf: Bayesian and hyperband scheduler"

wandb.login()

#### Run Binary classification 
1. Config

In [6]:
pretrain_model_path =  'BEHRT/Early_integration/Tasks/Output/Second_run/MLM_npi_final.pkl'   # pretrained MLM path

file_config = { 'data':'BEHRT/Early_integration/Files/data_preprocessed_1y_not_chunked_NPI.pkl',
               'index_to_delete': 'BEHRT/Early_integration/Files/index_to_delete.pkl',
                'labels' : 'BEHRT/Early_integration/Files/relapses.csv', 
                'test_idx' : 'BEHRT/Early_integration/Files/test_idx.pkl', 
                }


global_params = {
    'device': device,
    'output_dir': 'BEHRT/Early_integration/Tasks/Output/clf_npi', # output folder
    'best_name': 'clf_npi_3y', #'clf_quantiles_3',  # output model name
    'max_len_seq': 512, #100,
    'max_age': 110,
    'max_delay': 30, ## = 30years
    'age_month': 12,
    'delay_month': 0.25,
    'age_year': False,
    'age_symbol': None,
    'min_visit': 3,
    
}

train_params = {
    'use_cuda': True,
    'max_len_seq': global_params['max_len_seq'],
    'device': device,
}

feature_dict = {
    'word':True,
    'seg':True,
    'age':True,
    'modalities': True,
    'delays': True,
    'position': True, 
    'NPI' : True
}


2. Import data 

In [7]:
"""Import train data """
data =  hf._load_pkl(file_config['data'])

"""Import labels """
labels = hf._load_csv(file_config['labels'])
#labels.drop(['Unnamed: 0', 'status_drfs_5y'], axis='columns', inplace = True)
labels.rename(columns={'status_rfs_surg_3y': 'label'}, inplace=True)

"""Merge label and data """
data = pd.merge(data, labels, on=['Num_dossier'])
data.set_index('Num_dossier', inplace=True)

# Display 3 first rows
data.head(1)

,inputs_quantiles_preprocessed,inputs_normal_range_preprocessed,inputs_curve_intersections_preprocessed,modalities,age,delays,report,inputs_quantiles_preprocessed_10,inputs_normal_range_preprocessed_10,inputs_curve_intersections_preprocessed_10,...,status_rfs_5y,status_drfs_5y,status_rfs_surg_18mo,status_drfs_surg_18mo,status_rfs_surg_2y,status_drfs_surg_2y,status_rfs_surg_3y,status_drfs_surg_3y,label,status_drfs_surg_5y
Num_dossier,,,,,,,,,,,,,,,,,,,,,
00002f6df9cfcb97985c88f5787ba874,"[CLS, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[CLS, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, ...","[CLS, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[DEB, biologique, biologique, biologique, biol...","[61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 61, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[\nCorps du document\n\n\n\n\n\n\n\n\n\n16/07/...,"[CLS, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...","[CLS, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 0.0, 0.0, ...","[CLS, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, ...",...,0,0,0,0,0,0,0,0,0,0


In [8]:
## Delete censored patients and relapsed before 18months post surgery

index_to_del = hf._load_pkl(file_config['index_to_delete'])  # 0: 12mo, 1:18mo, 2:24mo, 3:36mo, 4:60mo
list_of_index = index_to_del['list_of_relapses_before_12mo']+index_to_del['list_of_censored_before_3y']
len(list_of_index)

10029

In [9]:
data = data.reset_index()
data = data.loc[~data['Num_dossier'].isin(list(list_of_index))] #I lost 3722 patients

In [11]:
# remove patients with visits less than min visit
previous_shape = data.shape[0]

data['length'] = data['inputs_normal_range_preprocessed_100_therap_removed'].apply(lambda x: len([i for i in range(len(x)) if x[i] == 'SEP']))
data = data[data['length'] >= global_params['min_visit']]
#data = data.reset_index(drop=True)

print("We lost {} patient".format(previous_shape - data.shape[0]))

We lost 0 patient


In [13]:
## use the vocabulary from the MLM

whole_seq = hf._load_pkl('BEHRT/Early_integration/Files/whole_seq_preprocessed_NPI.pkl')
tokenVocab, _ = input_vocab(inputs = whole_seq.inputs_normal_range_preprocessed_100_therap_removed, symbol=global_params['age_symbol'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], mon = global_params['age_month'], symbol=global_params['age_symbol'])
modalitiesVocab, _ = mod_vocab(whole_seq.modalities_100_therap_removed, symbol=global_params['age_symbol'])
delayVocab, _ = delay_vocab(max_delay=global_params['max_delay'], mon = global_params['delay_month'], symbol=global_params['age_symbol'])
NPIVocab = {'PAD':0, 'UNK':1, 1.0:2, 2.0:3, 3.0:4, 4.0:5, 5.0:6, 6.0:7}

del whole_seq

# Binary classification 
labelVocab = {0:0, 1:1}

4. Prepare data 

In [1]:
test_idx = hf._load_pkl(file_config['test_idx'])

NameError: name 'hf' is not defined

In [15]:
train = data.loc[data['Num_dossier'].isin([i for i in data['Num_dossier'] if i not in test_idx.values])]
test = data.loc[data['Num_dossier'].isin(test_idx)]

In [19]:
train['patid'] = range(len(train))
test['patid'] = range(len(test))

train.index =  range(len(train))
test.index =  range(len(test))

3. Config and utils

In [33]:
def load_model(path, model):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model

In [ ]:

## Config for MLM with sweep (wandb)

config = {'method' : 'bayes', 
                'metric' : {'name' : 'valid_f1', 'goal': 'maximize'},
                'parameters' : {'hidden_size': {'value': 432},       # word embedding and seg embedding hidden size
                'intermediate_size': {'value': 560},                # the size of the "intermediate" layer in the transformer encoder
                'hidden_dropout_prob': {'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},                    # dropout rate
                'num_hidden_layers': {'distribution': 'int_uniform', 'min':1, 'max':12},                   # number of multi-head attention layers required
                'num_attention_heads': {'values': [1, 3, 6, 9, 12]},                  # number of attention heads
                'attention_probs_dropout_prob': {'values': [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
                'hidden_act': {'value' :'gelu'},                                                 # The non-linear activation function in the encoder and the pooler "gelu", 'relu', 'swish' are supported
                'vocab_size': {'value': len(tokenVocab)},                                                  # number of disease + symbols for word embedding
                'seg_vocab_size': {'value':2},                                                             # number of vocab for seg embedding
                'modalities_vocab_size': {'value':len(modalitiesVocab)}, 
                'age_vocab_size': {'value':len(ageVocab)},                                                 # number of vocab for age embedding
                'delay_vocab_size': {'value':len(delayVocab)},
                'NPI_vocab_size' : {'value': len(NPIVocab)},
                'epochs': {'value': 100},
                'max_position_embeddings': {'value': train_params['max_len_seq']},                          # maximum number of tokens
                'initializer_range': {'value' : 0.02},                                                     # parameter weight initializer range
                'lr' : {'values': [1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3]},                      # a flat distribution between 0 and 0.1
                'warmup_proportion': {'value': 0.1}, 
                'weight_decay' : {'values': [0, 0.0001, 0.0005, 0.001, 0.005, 0.01,]}, 
                'class_weights': {'value': 1},
                'batch_size': {'values' : [16, 32, 64]},   # integers between 8 and 32 # with evenly-distributed logarithms
                'gradient_accumulation_steps': {'distribution': 'int_uniform', 'min':1, 'max':10},
                'chunk_size_feed_forward' : {'value':0},
                'max_position_embeddings': {'value' : train_params['max_len_seq']},                          # maximum number of tokens
                'output_attentions' : {'value':True},
                'is_decoder' : {'value':False},
                'add_cross_attention':{'value': False},
                'vocab_size': {'value':len(tokenVocab)},                                                  # number of disease + symbols for word embedding
                'seg_vocab_size':{'value':2},                                                             # number of vocab for seg embedding
                'modalities_vocab_size' :{'value':len(modalitiesVocab)}, 
                'age_vocab_size': {'value': len(ageVocab)},                                                 # number of vocab for age embedding
                'delays_vocab_size': {'value': len(delayVocab)},
                'layer_norm_eps' : {'value':  1e-5}},     
                'early_terminate': {'type': 'hyperband', 's': 2, 'eta': 3, 'max_iter': 27},
          
}

In [ ]:

class BertConfig(Bert.modeling.BertConfig):
    def __init__(self, config):
        super(BertConfig, self).__init__(
            vocab_size_or_config_json_file=config.get('vocab_size'),
            hidden_size=config['hidden_size'],
            num_hidden_layers=config.get('num_hidden_layers'),
            num_attention_heads=config.get('num_attention_heads'),
            intermediate_size=config.get('intermediate_size'),
            hidden_act=config.get('hidden_act'),
            hidden_dropout_prob=config.get('hidden_dropout_prob'),
            attention_probs_dropout_prob=config.get('attention_probs_dropout_prob'),
            max_position_embeddings = config.get('max_position_embeddings'),
            initializer_range=config.get('initializer_range'),
            
        )
        self.delays_vocab_size = config.get('delays_vocab_size')
        self.modalities_vocab_size = config.get('modalities_vocab_size')
        self.age_vocab_size = config.get('age_vocab_size')
        self.seg_vocab_size = config.get('seg_vocab_size')
        self.NPI_vocab_size = config.get('NPI_vocab_size')
        self.output_attentions = config.get('output_attentions')
        self.chunk_size_feed_forward = config.get('chunk_size_feed_forward')
        self.is_decoder = config.get('is_decoder')
        self.layer_norm_eps = config.get('layer_norm_eps')
        self.add_cross_attention = config.get('add_cross_attention')
  

In [34]:
def define_model(config):
    
    # del model
    
    class_weights = torch.FloatTensor([1,config.class_weights]).cuda()
    conf = BertConfig(config)
    model = BertForClassification(conf, num_labels=len(labelVocab.keys()), feature_dict=feature_dict, class_weights = class_weights)
    model = load_model(pretrain_model_path, model)   
    model = model.to(global_params['device'])
    optim = optimiser.adam(params=list(model.named_parameters()), config=config)
 
    return model, optim

In [ ]:

sweep_id = wandb.sweep(config, project='Binary_clf_npi_3y_run2')


#### Model 
1. Utils



In [35]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=list(labelVocab.values()))
mlb.fit([[each] for each in list(labelVocab.values())])

MultiLabelBinarizer(classes=[0, 1])

In [36]:
## Scores 

import sklearn
def precision(logits, label):

    label, logits = label.cpu(), logits.detach().cpu()
    label=np.argmax(label, axis=1)
    logits=np.argmax(logits, axis=1)
    f1 =  sklearn.metrics.f1_score(label.numpy(), logits.numpy(), average="macro")
    
    return f1, logits, label

def precision_valid(logits, label):
    label=np.argmax(label, axis=1)
    logits=np.argmax(logits, axis=1)   
    f1 =  sklearn.metrics.f1_score(label.numpy(), logits.numpy(), average="macro")
    roc = sklearn.metrics.roc_auc_score(label.numpy(),logits.numpy(), average='macro')
    
    
    return f1, roc, logits, label



def precision_test(output, label):
    label=np.argmax(label, axis=1)#.flatten().numpy()
    output=np.argmax(output, axis=1)#.flatten().numpy()
    roc = sklearn.metrics.roc_auc_score(label ,output, average='macro')
    f1 =  sklearn.metrics.f1_score(label, output, average="macro")
    recall = sklearn.metrics.recall_score(label, output, average="macro")
    precision = sklearn.metrics.precision_score(label, output, average="macro")
 
    return f1, roc, recall, precision, output, label


def test_scores(model, data): 
     
    y = []
    y_label = []
    total_f1, total_roc, total_precision, total_recall = [], [], [], []
    model.eval()


    for step, batch in enumerate(data):

        age_ids, input_ids, mod_ids, del_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
       
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)

        age_ids = age_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        del_ids = del_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])

        with torch.no_grad():
            outputs = model(input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, NPI_ids, attention_mask=attMask, labels=targets, output_attentions=True)
        

        logits = outputs.logits.cpu()
        targets = targets.cpu()

        y_label.append(targets)
        y.append(logits)
        
    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0)

     # Compute ROC curve and ROC area for each class
    f1, roc, recall, precision, output, label = precision_test(y, y_label)

    # Compute micro-average ROC curve and ROC area
    label = np.argmax(y_label, axis=1)
    sig = nn.Sigmoid()
    output=sig(y)

    fpr, tpr, _ = sklearn.metrics.roc_curve(label, output[:,1])
    roc_auc = sklearn.metrics.auc(fpr, tpr)

    return f1, recall, precision, fpr, tpr, roc_auc

In [37]:

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(timedelta(seconds=elapsed_rounded))

training_stats = []

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    
   
def training_per_epoch(e, loader, validloader, model , optim, config):
    
    start = time.time()
    
    # Reset the loss at each epoch
    temp_loss, temp_f1 = [], []
    train_f1, train_loss = [], []
    nb_tr_examples, nb_tr_steps = 0, 0
    train_targets, train_masks = [], []
    
    ## Count the number of batches
    cnt = 0
    count = 0
    # Training
    model.train()
    

    for step, batch in enumerate(loader):
        
        cnt +=1
        age_ids, input_ids, mod_ids, del_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
 
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)
    
        ## Load batch to gpu 
        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        del_ids = del_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        
        ## Compute output (loss, logits and attentions scores) 
        output = model(input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, NPI_ids, attention_mask=attMask, labels=targets, output_attentions = True)
      
        loss = output.loss
        logits  = output.logits
        attentions = output.attentions
        
        if config.gradient_accumulation_steps >1:
            loss = loss/config.gradient_accumulation_steps
        

        temp_loss.append(loss.item())
        train_loss.append(loss.item())
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        f1, a, b = precision(logits, targets)
       
        temp_f1.append(f1)
   

        
        if (step + 1) % config.gradient_accumulation_steps == 0:
            optim.zero_grad()
            loss.backward()
            optim.step()
            
        train_f1.append(f1)
        count+=1
        
        train_targets.append(targets)
        train_masks.append(attMask)
        logits = logits.cpu()
        targets = targets.cpu()
        loss = loss.cpu()
        torch.cuda.empty_cache()

    train_hidden_states = output.hidden_states[1:]

        
    # Measure how long this epoch took.
    training_time = format_time(time.time() - start)
                   
    print("")
    print("Average training loss: {0:.2f}".format(np.mean(train_loss)))
    print("Average training score: {0:.2f}".format(np.mean(train_f1)))
    print("Training epoch took: {:}".format(training_time))
    
          
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")
    
    valid_f1, valid_roc, valid_loss, valid_time, valid_hidden_states, valid_targets, valid_masks = evaluate(validloader, model)
    
    # Record all statistics from this epoch.
    training_stats.append({'epoch': e+1, 
                               'Training Loss': np.mean(train_loss) ,
                               'Training Time' : training_time,
                               'Average Train f1 score' : np.mean(train_f1),
                               'Validation Loss': np.mean(valid_loss), 
                               'Validating Time' : valid_time, 
                               'Average Valid f1 score' : np.mean(valid_f1),
                               'Average Valid ROC' : np.mean(valid_roc)
                                })  
        
    return valid_f1, train_f1, valid_loss, train_loss, valid_roc, training_stats, train_hidden_states, valid_hidden_states, train_targets, valid_targets, train_masks, valid_masks
    

def evaluate(loader, model):
                               
    start = time.time()
                               
    model.eval()
    y = []
    y_label = []
    val_loss, total_f1, total_roc = [], [], []
    val_loss_tot, tot_f1, tot_roc = [], [], []
    targets_list, masks = [], []
    
    for step, batch in enumerate(loader):
        
        age_ids, input_ids, mod_ids, del_ids, NPI_ids, posi_ids, segment_ids, attMask, targets, _ = batch
 
        targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)
    
        ## Load batch to gpu 
        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        mod_ids = mod_ids.to(global_params['device'])
        del_ids = del_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        NPI_ids = NPI_ids.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
      
        with torch.no_grad():
            outputs = model(input_ids, mod_ids, age_ids, del_ids, segment_ids, posi_ids, NPI_ids, attention_mask=attMask, labels=targets, output_attentions=True)
        
        val_loss.append(outputs.loss.item())
        
        logits = outputs.logits.cpu()
        targets = targets.cpu()
        
        y_label.append(targets)
        y.append(logits)
        
        masks.append(attMask)
        torch.cuda.empty_cache()
        
    hidden_states = outputs.hidden_states[1:]
    targets_list = y_label
    
    
    y_label = torch.cat(y_label, dim=0)
    y = torch.cat(y, dim=0) 
    
    tot_f1, tot_roc, output, label = precision_valid(y, y_label)
   
    # Measure how long this epoch took.
    validation_time = format_time(time.time() - start)
    
    return tot_f1, tot_roc, val_loss, validation_time, hidden_states, targets_list, masks

def training(config=None):
    
    with wandb.init(config=config) as run:
        
        config = wandb.config
        
        train_hidden_states_per_epoch, train_targets_per_epoch, train_mask_per_epoch =  [] , [], []
        valid_hidden_states_per_epoch, valid_targets_per_epoch, valid_mask_per_epoch = [], [], []  
        train_results, valid_results, valid_results_tot, train_loss, valid_loss, valid_results_roc, training_stats_list  = [], [], [], [], [], [], []
        train_f1_list, train_loss_list = [], []
        valid_f1_list, valid_f1_list_tot, valid_loss_list, valid_roc_list = [], [], [], []
        best_pre = 0.0
        
        X_train, X_valid, y_train, y_valid = train_test_split(train.drop(['label'], axis=1), train['label'], test_size=0.1, random_state=42, stratify=train['label'])
        
        train_df = pd.concat([X_train, y_train], axis=1)
        cv_df = pd.concat([X_valid, y_valid], axis=1)
      
            
        train_df.index = range(train_df.shape[0])
        train_df.patid = range(train_df.shape[0])
        X_tr = train_df['inputs_normal_range_preprocessed_100_therap_removed']
        y_tr = train_df['label']

        cv_df.index = range(cv_df.shape[0])
        cv_df.patid = range(cv_df.shape[0])

        train_loader = load_data(train_df, config, X_tr=X_tr, y_tr=y_tr)
        valid_loader = load_data(cv_df, config, sampling=False)

        model, optim = define_model(config)

        for e in range(config.epochs):   

            valid_f1, train_f1, valid_loss, train_loss, valid_roc, training_stats, train_hidden_states, valid_hidden_states, train_targets, valid_targets, train_masks, valid_masks =  training_per_epoch(e, train_loader, valid_loader, model, optim, config)       

            mean_f1 = np.mean(valid_f1)

            if mean_f1 > best_pre:
                print("** ** * Saving fine - tuned model ** ** * ")
                model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
                output_model_file = os.path.join(global_params['output_dir'],global_params['best_name'])
                create_folder(global_params['output_dir'])

                torch.save(model_to_save.state_dict(), output_model_file)
                best_pre = mean_f1

            wandb.log({'f1' : np.mean(train_f1), 'loss': np.mean(train_loss), 'valid_f1': valid_f1, 'valid_roc':valid_roc,
                      'valid_loss':np.mean(valid_loss), 'best_f1':best_pre})
        
        test.index = range(test.shape[0])
        test.patid = range(test.shape[0])
        test_loader = load_data(test, config, sampling=False)
        test_f1, test_recall, test_precision, fpr, tpr, test_roc  = test_scores(model, test_loader)
    #    test_f1, test_roc, test_loss, test_time, test_hidden_states, test_targets, test_masks = evaluate(test_loader, model)
        wandb.log({'test_f1':test_f1, 'test_roc': test_roc, 'test_recall': test_recall, 'test_precision':test_precision})
        
        art = wandb.Artifact(f'clf_npi_3y_{wandb.run.id}', type='model')
        art.add_file(output_model_file)
        run.log_artifact(art)
        
        if test_f1 >= 0.65:
            wandb.alert(
            title="Higher f1", 
            text=f"3years: You reach your higher f1: {test_f1}",
            )
            torch.save(model_to_save.state_dict(), 'BEHRT/Early_integration/Tasks/Output/clf_npi/Above_baselines_3y')
  
        wandb.finish()
        

2. Training

In [40]:

if __name__=="__main__":
    

   ## Run hyperparameters tuning 5times for the training
    start = time.time()
    
    wandb.agent(sweep_id, training, count=50)
    
    print()
    print('Elapsed time for sweep hyperparameters tuning = {}"'.format(time.time() - start))
    



Generating Inputs for fold 0


t_total value of -1 results in schedule not being applied


epoch: 0	| Cnt: 1	| Loss: 0.5719892978668213	| f1 score: 0.5465587044534412

Average training loss: 0.31
Average training score: 0.34
Training epoch took: 0:00:09

Running Validation...
Validation Loss: 1.44
Validation took: 0:00:03

** ** * Saving fine - tuned model ** ** * 
valid score: 0.09651307596513076

epoch: 1	| Cnt: 1	| Loss: 0.2754223048686981	| f1 score: 0.3333333333333333

Average training loss: 0.27
Average training score: 0.33
Training epoch took: 0:00:09

Running Validation...
Validation Loss: 1.40
Validation took: 0:00:03

valid score: 0.09651307596513076

epoch: 2	| Cnt: 1	| Loss: 0.2739230990409851	| f1 score: 0.3333333333333333

Average training loss: 0.27
Average training score: 0.33
Training epoch took: 0:00:10

Running Validation...
Validation Loss: 1.40
Validation took: 0:00:03

valid score: 0.09651307596513076

epoch: 3	| Cnt: 1	| Loss: 0.2679360508918762	| f1 score: 0.3333333333333333

Average training loss: 0.27
Average training score: 0.35
Training epoch took

OutOfMemoryError: CUDA out of memory. Tried to allocate 96.00 MiB (GPU 0; 31.75 GiB total capacity; 29.86 GiB already allocated; 83.69 MiB free; 30.53 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF